In [57]:
import ccxt
import pprint
import pandas as pd

import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

with open("./api.txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    api_secret = lines[1].strip()

exchange = ccxt.binanceusdm({
    'apiKey': api_key,
    'secret': api_secret,
})



In [58]:
symbol = "BTC/USDT"
order_book = exchange.fetch_order_book(symbol=symbol)
pprint.pprint(order_book)

{'asks': [[23077.0, 0.253],
          [23077.3, 0.001],
          [23077.5, 0.001],
          [23077.7, 0.009],
          [23077.9, 0.001],
          [23078.0, 0.435],
          [23078.1, 0.261],
          [23078.3, 0.001],
          [23078.5, 3.304],
          [23078.6, 10.252],
          [23078.7, 0.001],
          [23078.8, 0.015],
          [23078.9, 0.008],
          [23079.0, 0.001],
          [23079.1, 0.002],
          [23079.2, 0.001],
          [23079.3, 0.003],
          [23079.5, 0.002],
          [23079.6, 0.002],
          [23079.7, 0.045],
          [23079.9, 0.647],
          [23080.0, 1.49],
          [23080.1, 0.651],
          [23080.2, 0.001],
          [23080.3, 0.003],
          [23080.4, 2.214],
          [23080.5, 3.958],
          [23080.6, 0.101],
          [23080.7, 0.214],
          [23080.8, 4.264],
          [23080.9, 5.675],
          [23081.0, 1.334],
          [23081.1, 0.357],
          [23081.2, 0.049],
          [23081.3, 0.257],
          [23081.4, 

In [59]:
order_book_df= pd.DataFrame(columns=['asks','bids','timestamp'])
order_book_df

,asks,bids,timestamp


In [60]:
asks_data = order_book['asks'][:10]
bids_data = order_book['bids'][:10]
datetime_data = order_book['datetime']
datetime_data = datetime.datetime.strptime(datetime_data,'%Y-%m-%dT%H:%M:%S.%fZ')
datetime_data = datetime_data.strftime('%Y-%m-%d %H:%M:%S.%f')
order_book_df.append({'asks':asks_data,'bids':bids_data,'timestamp':datetime_data},ignore_index=True)

,asks,bids,timestamp
0,"[[23077.0, 0.253], [23077.3, 0.001], [23077.5,...","[[23076.9, 17.031], [23076.8, 0.684], [23076.7...",2023-02-01 02:19:48.908000


In [38]:
import datetime
import pytz
import time

def timestamp_to_date(mili_time):
    utc_dt = datetime.datetime.utcfromtimestamp(mili_time / 1000.0)
    utc_dt = pytz.utc.localize(utc_dt) # utc_df가 utc시 시간이라는것을 

    korea = pytz.timezone('Asia/Seoul')
    korea_dt = korea.normalize(utc_dt.astimezone(korea))

    timeline = str(korea_dt.strftime('%Y-%m-%d %H:%M:%S'))  #(1) 출력형식 지정
    return timeline

print(timestamp_to_date(1675215359888))


def date_to_timestamp(date,utc=False):
    """
    str형태의 date를 timestamp로 만들어주기
    :params (str or datetime) date : '%Y-%m-%d %H:%M:%S'형태의 데이터. ex)'2023-01-18 23:00:00'
    :params bool utc : True로 설정할시에 date를 utc 시간이라고 생각
    :return timestamp시간(단위 ms) ex)1674050400000
    :rtype int
    
    ex) date_to_timestamp('2023-01-18 23:00:00') -> 1674050400000
    """
    if type(date) == str: # str인 경우 datetime으로 변환해주기
        dt = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M:%S')
    else: # datetime으로 들어온경우
        dt = date
        
    # time.mktime은 local 타임 기준으로 timestamp를 변경함. 따라서 utc일 경우 +9시간해서 한국시간으로 설정해줘야함
    if utc:
        dt = date + datetime.timedelta(hours = 9)
    
    ts = time.mktime(dt.timetuple()) 
    
    return int(ts*1000)

print(date_to_timestamp('2023-02-01 10:35:59'))

2023-02-01 10:35:59
1675215359000


In [67]:
def make_order_book(coin_name,start_time,end_time):
    """
    coin이름 , 수집 시작시간, 수집 종료 시간 을 입력하면 orderbook이 그 시간 기준으로 수집됨
    :params str coin_name : 코인이름 ex)"BTC/USDT"
    :params str start_time : 수집 시작 시간 ex) '2022-01-01 00:00:00' (한국시간기준)
    :params str end_time : 수집 끝 시간 ex) '2023-01-01 00:00:00' (한국시간기준)
    :return None
    :rtype None
    """
    
    start_timestamp = date_to_timestamp(start_time)
    end_timestamp = date_to_timestamp(end_time)
    
    exchange = ccxt.binanceusdm()
    
    order_book_df= pd.DataFrame(columns=['asks','bids','datetime'])

    while True:
        order_book = exchange.fetch_order_book(symbol=coin_name)
        asks_data = order_book['asks'][:10]
        bids_data = order_book['bids'][:10]
        
        datetime_data = order_book['datetime']
        datetime_data = datetime.datetime.strptime(datetime_data,'%Y-%m-%dT%H:%M:%S.%fZ')
        datetime_data = datetime_data.strftime('%Y-%m-%d %H:%M:%S.%f')
        
        timestamp_data = order_book['timestamp']
        
        time.sleep(0.4) # 약 0.5초마다 변경됨 0.4초로 설정하고 나중에 중복되는 데이터 제거해주면 더 정확하게 수집됨

        if timestamp_data<start_timestamp:
            continue
        
        if timestamp_data>end_timestamp:
            break
        
        order_book_df = order_book_df.append({'asks':asks_data,'bids':bids_data,'datetime':datetime_data},ignore_index=True)
    
    order_book_df.set_index('datetime',inplace=True)
    
    coin_name = "".join(coin_name.split("/"))
    s_time = "-".join("-".join(str(start_time).split(" ")).split(":"))
    e_time = "_".join("-".join(str(end_time).split(" ")).split(":"))
    order_book_df.to_csv(f'./orderbook_{coin_name}_{s_time}_{e_time}.csv')

In [68]:
make_order_book('BTC/USDT','2023-02-01 11:26:00','2023-02-01 11:27:00')